In [1]:
import json
from pathlib import Path

path = Path("../data/raw/season1.json")
raw = path.read_text(encoding="utf-8")
data = json.loads(raw)

type(data), len(data)

(dict, 10)

In [6]:
episodes = list(data.keys())
episodes

first_episode = data[episodes[0]]
first_episode

list(first_episode.items())[:5]

[('1', 'Easy, boy.'),
 ('10', 'Our orders were to track the wildlings.'),
 ('100', '- Right. Give it here. - No!'),
 ('101', 'Put away your blade.'),
 ('102', '- I take orders from your father, not you. - Please, Father!')]

In [8]:
!pip install pandas
import pandas as pd

rows = []
for episode_name, subtitles in data.items():
    for subtitle_id, text in subtitles.items():
        rows.append({
            "episode": episode_name,
            "subtitle_id": int(subtitle_id),
            "text": text
        })

df = pd.DataFrame(rows).sort_values(["episode", "subtitle_id"]).reset_index(drop=True)
df.head()

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 44.5 MB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 56.9 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]━━━━━━━━━━━ 2/3 [pandas]


,episode,subtitle_id,text
0,Game Of Thrones S01E01 Winter Is Coming.srt,1,"Easy, boy."
1,Game Of Thrones S01E01 Winter Is Coming.srt,2,What do you expect? They're savages.
2,Game Of Thrones S01E01 Winter Is Coming.srt,3,"One lot steals a goat from another lot,"
3,Game Of Thrones S01E01 Winter Is Coming.srt,4,before you know it they're ripping each other ...
4,Game Of Thrones S01E01 Winter Is Coming.srt,5,I've never seen wildlings do a thing like this.


In [9]:
import re

def clean_episode(name):
    match = re.search(r"S\d+E\d+", name)
    return match.group(0) if match else name

df["episode"] = df["episode"].apply(clean_episode)
df.head()


,episode,subtitle_id,text
0,S01E01,1,"Easy, boy."
1,S01E01,2,What do you expect? They're savages.
2,S01E01,3,"One lot steals a goat from another lot,"
3,S01E01,4,before you know it they're ripping each other ...
4,S01E01,5,I've never seen wildlings do a thing like this.


In [10]:
import re

def tokenize(text):
    return re.findall(r"[A-Za-z']+", text.lower())

df["tokens"] = df["text"].apply(tokenize)
df[["episode", "subtitle_id", "tokens"]].head()


,episode,subtitle_id,tokens
0,S01E01,1,"[easy, boy]"
1,S01E01,2,"[what, do, you, expect, they're, savages]"
2,S01E01,3,"[one, lot, steals, a, goat, from, another, lot]"
3,S01E01,4,"[before, you, know, it, they're, ripping, each..."
4,S01E01,5,"[i've, never, seen, wildlings, do, a, thing, l..."


In [11]:
from itertools import chain

all_tokens = list(chain.from_iterable(df["tokens"]))
len(all_tokens)


44935